# **Cálculo de la distancia media entre señales exponenciales complejas Ejercicio #1**

Dadas las señales:
$$
x_1(t) = A e^{-j n \omega_0 t}, \quad x_2(t) = B e^{j m \omega_0 t}
$$

La **distancia cuadrática media** entre ellas se define como:
$$
d^2(x_1, x_2) = \lim_{T \to \infty} \frac{1}{T} \int_T |x_1(t) - x_2(t)|^2 dt
$$

Desarrollaremos esta expresión numéricamente en Python, considerando distintos valores de
$A$, $B$, $n$, $m$, y $\omega_0$.

Usaremos integración numérica en un intervalo suficientemente largo para aproximar el promedio temporal.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Usaremos numpy para operaciones matemáticas (senos, exponenciales, etc.)
# y matplotlib para visualizar los resultados.

# **Definición de parámetros**

Definimos los parámetros principales del problema:

- $A$ y $B$: amplitudes de las señales.
- $n$ y $m$: índices armónicos.
- $\omega_0$: frecuencia angular fundamental.
- $t$: vector de tiempo para la integración numérica.

Usaremos un intervalo suficientemente grande para simular el límite cuando
$T \to \infty$.


In [ ]:
A = 2       # Amplitud de la primera señal
B = 3       # Amplitud de la segunda señal
n = 2       # Índice armónico de la primera señal
m = -2      # Índice armónico de la segunda señal
w0 = 2 * np.pi * 100  # Frecuencia angular fundamental (rad/s)

# Definimos el tiempo para observar varios periodos
T0 = 2 * np.pi / w0
t = np.linspace(0, 50*T0, 50000)  # Ventana larga de tiempo

# **Definición de las señales $x_1(t)$ y $x_2(t)$**

De acuerdo con el enunciado:
$$
x_1(t) = A e^{-j n \omega_0 t}, \quad x_2(t) = B e^{j m \omega_0 t}
$$

En Python usamos números complejos con `1j` para representar $j = \sqrt{-1}$.


In [ ]:
x1 = A * np.exp(-1j * n * w0 * t)
x2 = B * np.exp(1j * m * w0 * t)

# **Cálculo numérico de la distancia media**

La distancia media se define como:
$$
d^2 = \frac{1}{T} \int_T |x_1(t) - x_2(t)|^2 dt
$$

Aproximamos la integral con la función `np.trapz()` (regla del trapecio), dividiendo por el intervalo total.


In [ ]:
diff = np.abs(x1 - x2)**2           # Diferencia al cuadrado
d2 = np.trapz(diff, t) / (t[-1] - t[0])  # Integral promedio (distancia cuadrática media)
d = np.sqrt(d2)                     # Distancia media
print(f"Distancia media d(x1,x2) = {d:.4f}")


# **Comparación con la solución teórica**

Según el análisis:
$$
d(x_1, x_2) =
\begin{cases}
\sqrt{A^2 + B^2}, & m \neq -n, \\
|A - B|, & m = -n
\end{cases}
$$

Verificaremos si el resultado numérico coincide con el valor teórico.


In [ ]:
if m == -n:
    d_teorico = abs(A - B)
else:
    d_teorico = np.sqrt(A**2 + B**2)

print(f"Distancia teórica: {d_teorico:.4f}")
print(f"Error relativo: {abs(d - d_teorico)/d_teorico*100:.6f}%")


# **Muestreo y Cuantización de una Señal Analógica Ejercicio #2**

En este experimento simularemos el proceso de **muestreo y cuantización uniforme** de una señal analógica compuesta por varias componentes senoidales.  

Se analiza paso a paso:

1. Definición de parámetros de muestreo y cuantización.  
2. Generación de la señal continua y muestreada.  
3. Aplicación de un cuantizador uniforme tipo *mid-tread*.  
4. Visualización comparativa de las tres señales:
   - $$x(t)$$ → señal continua  
   - $$x[n]$$ → señal muestreada  
   - $$x_q[n]$$ → señal cuantizada


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


fs = 5000            # Frecuencia de muestreo (Hz)
Ts = 1 / fs          # Período de muestreo (s)
bits = 4             # Resolución del ADC (número de bits)
L = 2**bits          # Número total de niveles de cuantización

# Tiempo continuo: simulamos 6 ms (≈3 periodos)
t_cont = np.linspace(0, 0.006, 10000)

# Tiempo discreto: en pasos de Ts
t_disc = np.arange(0, 0.006, Ts)

# Señal analógica original: combinación de tres senoidales
x_t = 3*np.cos(1000*np.pi*t_cont) + 5*np.sin(3000*np.pi*t_cont) + 10*np.cos(11000*np.pi*t_cont)

# Señal muestreada
x_s = 3*np.cos(1000*np.pi*t_disc) + 5*np.sin(3000*np.pi*t_disc) + 10*np.cos(11000*np.pi*t_disc)

- **Frecuencia de muestreo ($f_s$):** 5000 Hz.  
  Esto significa que se toman 5000 muestras por segundo.  

- **Periodo de muestreo ($T_s$):**
  $$
  T_s = \frac{1}{f_s}
  $$

- **Resolución del ADC:**  
  Con 4 bits, se obtienen:
  $$
  L = 2^{bits} = 16 \text{ niveles de cuantización}
  $$

- La señal $$x(t)$$ está compuesta por tres frecuencias diferentes, lo que produce una forma de onda compleja.  
- Se crea también la versión discreta $$x[n]$$ tomando muestras en intervalos de $$T_s$$


In [ ]:
Xmax = 18                   # Rango máximo esperado de la señal
Delta = 2*Xmax / (L - 1)    # Paso de cuantización
niveles = np.linspace(-Xmax, Xmax, L)  # Vector con los niveles

# Función de cuantización
def cuantizar(x):
    # Redondea al múltiplo más cercano de Δ
    x_q = Delta * np.round(x / Delta)
    # Limita los valores dentro del rango permitido
    x_q = np.clip(x_q, -Xmax, Xmax)
    return x_q

# Aplicamos cuantización a la señal muestreada
x_q = cuantizar(x_s)

La cuantización convierte los valores continuos de la señal muestreada en niveles discretos.

1. **Paso de cuantización (Δ):**
   $$
   \Delta = \frac{2 X_{max}}{L - 1}
   $$

2. **Cuantización tipo mid-tread:**  
   Se redondea el valor más cercano al múltiplo de Δ (el nivel 0 tiene una región muerta alrededor).

3. **Saturación:**  
   Con `np.clip()` se evita que los valores sobrepasen el rango ±Xmax.

El resultado es una señal $$x_q[n]$$ con valores discretos, lista para digitalización.


In [ ]:
print(f"Frecuencia de muestreo: {fs} Hz")
print(f"Niveles de cuantización: {L}")
print(f"Paso de cuantización (Δ): {Delta:.2f}")
print(f"Máximo valor de señal: {np.max(np.abs(x_s)):.2f}")
print(f"Rango: ±{Xmax}")

Estos valores permiten verificar que los parámetros sean coherentes:

- La **frecuencia de muestreo** define cuántas muestras se toman por segundo.  
- El **número de niveles (L)** determina la precisión del ADC.  
- El **paso de cuantización (Δ)** indica la distancia entre dos niveles consecutivos.  
- El **rango máximo (±Xmax)** evita saturación de la señal.  
Si la amplitud real de la señal supera ±Xmax, se produce **recorte** (clipping).

In [ ]:
# ==============================
# GRAFICAR
# ==============================
plt.figure(figsize=(10,6))

# Señal continua
plt.plot(t_cont*1000, x_t, label='x(t) continua', color='gray', linewidth=1)

# Señal muestreada
plt.stem(t_disc*1000, x_s, linefmt='C0-', markerfmt='C0o', basefmt=" ", label='x[n] muestreada')

# Señal cuantizada
plt.stem(t_disc*1000, x_q, linefmt='C1--', markerfmt='C1s', basefmt=" ", label='x_q[n] cuantizada')

plt.xlabel('Tiempo [ms]')
plt.ylabel('Amplitud')
plt.title('Muestreo y cuantización de x(t)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


En la figura se observan tres curvas:

1. **x(t):** la señal original continua (gris), de apariencia suave.  
2. **x[n]:** la señal muestreada (círculos azules).  
3. $x_q[n]$
 la señal cuantizada (cuadrados naranjas), con niveles discretos.

Se aprecia cómo la cuantización **reduce la resolución** de la señal al aproximarla a valores fijos, introduciendo el **error de cuantización**.


# **Ejercicio #3**

En este experimento se calcula y analiza la **Serie de Fourier** de una señal **periódica y simétrica** tipo triangular.

Se comparan dos métodos:
1. **Método analítico (doble derivada)** → cálculo exacto de los coeficientes.  
2. **Método numérico (integración)** → aproximación mediante sumas discretas.

Además, se grafican:
- La señal original $$x(t)$$  
- La **magnitud del espectro** $$|X[n]|$$  
- La **fase del espectro** $$\phi_n$$  

Finalmente, se calcula el **error relativo** entre el método exacto y el numérico.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 1.0       # Amplitud máxima de la señal
T = 10.0      # Periodo fundamental
d1 = 1.0      # Primer punto de cambio de pendiente
d2 = 2.0      # Segundo punto de cambio de pendiente

omega0 = 2 * np.pi / T   # Frecuencia angular fundamental

# Rango de índices n a calcular
n_values = np.arange(-5, 6)  # Desde n = -5 hasta n = 5

# Parámetros para integración numérica
N_samples = 4096              # Número de muestras para discretización
dt = T / N_samples
t_num = np.linspace(-T/2, T/2 - dt, N_samples)

- La señal es periódica con periodo $$T = 10 \, s$$ y amplitud máxima $$A = 1$$
- Los puntos $$d_1$$ y $$d_2$$ determinan los cambios de pendiente dentro del periodo.  
- La frecuencia fundamental se define como:
  $$
  \omega_0 = \frac{2\pi}{T}
  $$

Se usarán **4096 muestras** para aproximar la integral continua al calcular los coeficientes de Fourier numéricamente.

In [ ]:
def x_t(t, A, T, d1, d2):
    """
    Define la señal x(t) periódica dentro de un solo periodo [-T/2, T/2).
    Utiliza np.mod() para manejar la periodicidad.
    """
    # Mapeo del tiempo al intervalo fundamental [-T/2, T/2)
    t_mod = (t + T/2) % T - T/2
    x = np.zeros_like(t_mod)

    # 1️⃣ Rampa positiva (0 a d1)
    mask_pos1 = (t_mod >= 0) & (t_mod < d1)
    x[mask_pos1] = A * t_mod[mask_pos1] / d1

    # 2️⃣ Rampa negativa (d1 a d2)
    mask_neg2 = (t_mod >= d1) & (t_mod < d2)
    x[mask_neg2] = -A * (t_mod[mask_neg2] - d2) / (d2 - d1)

    # 3️⃣ Rampa negativa simétrica (-d1 a 0)
    mask_neg1 = (t_mod >= -d1) & (t_mod < 0)
    x[mask_neg1] = -A * t_mod[mask_neg1] / d1

    # 4️⃣ Rampa positiva simétrica (-d2 a -d1)
    mask_pos2 = (t_mod >= -d2) & (t_mod < -d1)
    x[mask_pos2] = A * (t_mod[mask_pos2] + d2) / (d2 - d1)

    return x

La función `x_t()` construye un **período de la señal triangular** a partir de cuatro segmentos lineales:

1. **Rampa ascendente** de 0 a $$d_1$$
2. **Rampa descendente** de $$d_1$$ a $$d_2$$  
3. **Rampa descendente simétrica** de $$-d_1$$ a 0  
4. **Rampa ascendente simétrica** de $$-d_2$$ a $$-d_1$$  

La función `np.mod()` garantiza que el tiempo siempre se mapee dentro de un periodo, haciendo que la señal sea periódica.


In [ ]:
def X_n_exact(n, T, A, d1, d2):
    """
    Calcula los coeficientes X[n] usando la fórmula exacta del método de la doble derivada.
    """
    omega0 = 2 * np.pi / T
    X_n = np.zeros_like(n, dtype=complex)

    # Caso n = 0 (componente DC)
    mask_n0 = (n == 0)
    X_n[mask_n0] = A * d2 / T

    # Caso n ≠ 0
    mask_nn0 = (n != 0)
    n_nn0 = n[mask_nn0]

    term1_num = np.cos(n_nn0 * omega0 * d2)
    term1_den = d2 - d1

    term2_num = d2 * np.cos(n_nn0 * omega0 * d1)
    term2_den = d1 * (d2 - d1)

    term3 = 1 / d1

    X_double_prime_scaled = term1_num / term1_den - term2_num / term2_den + term3

    factor = -2 * A / (T * (n_nn0 * omega0)**2)
    X_n[mask_nn0] = factor * X_double_prime_scaled

    return X_n

El **método de la doble derivada** permite obtener una expresión cerrada para los coeficientes de Fourier.

Para $$n \neq 0$$
$$
X[n] = -\frac{2A}{T(n\omega_0)^2}\left[\frac{\cos(n\omega_0 d_2)}{d_2 - d_1} - \frac{d_2 \cos(n\omega_0 d_1)}{d_1(d_2 - d_1)} + \frac{1}{d_1}\right]
$$

Y para $$n = 0$$
$$
X[0] = \frac{A d_2}{T}
$$

Esto evita integrar directamente la señal y reduce el costo computacional.


In [ ]:
def X_n_numerical(n, T, x_t_func, t_num, dt):
    """
    Calcula X[n] mediante integración numérica (suma de Riemann).
    """
    omega0 = 2 * np.pi / T
    X_n = np.zeros_like(n, dtype=complex)
    x_sampled = x_t_func(t_num, A, T, d1, d2)

    for k, n_k in enumerate(n):
        integrand = x_sampled * np.exp(-1j * n_k * omega0 * t_num)
        integral = np.sum(integrand) * dt
        X_n[k] = integral / T

    return X_n

Esta función implementa la **definición clásica** del coeficiente de Fourier:
$$
X[n] = \frac{1}{T}\int_{-T/2}^{T/2} x(t)e^{-j n \omega_0 t} dt
$$

La integral se aproxima por una suma discreta:
$$
\int f(t)dt \approx \sum f(t_i)\Delta t
$$

Esto permite comparar la exactitud del método analítico frente al numérico.


In [ ]:
X_exact = X_n_exact(n_values, T, A, d1, d2)
X_numerical = X_n_numerical(n_values, T, x_t, t_num, dt)

# Cálculos adicionales
X_exact_real = np.real(X_exact)
X_exact_imag = np.imag(X_exact)
X_exact_magn = np.abs(X_exact)
X_exact_phase = np.angle(X_exact)

# Error relativo
error_relativo = np.abs(X_exact_magn - np.abs(X_numerical)) / np.abs(np.abs(X_numerical))
error_relativo[np.abs(X_numerical) == 0] = 0

# Mostrar tabla
print(f"--- PARÁMETROS: A={A}, T={T}, d1={d1}, d2={d2}, ω₀={omega0:.2f} rad/s ---")
print("\n--- Espectro de Fourier y Componentes ---")
print("n | Re{X[n]} | Im{X[n]} | |X[n]| | Fase (rad) | Error Relativo (%)")
print("-----------------------------------------------------------------------")
for i in range(len(n_values)):
    print(f"{n_values[i]:2d} | {X_exact_real[i]:8.5f} | {X_exact_imag[i]:8.5e} | {X_exact_magn[i]:6.5f} | {X_exact_phase[i]:10.5f} | {error_relativo[i]*100:10.4f}")

Aquí se imprimen los resultados para cada armónico $$n$$

- Parte **real e imaginaria** de $$X[n]$$  
- **Magnitud** y **fase** del espectro.  
- **Error relativo (%)** entre la estimación exacta y la numérica.

Debido a la simetría de la señal, los coeficientes deben ser **reales** (la parte imaginaria será ≈0).


In [ ]:
plt.figure(figsize=(14, 10))

# 1️⃣ Señal x(t)
plt.subplot(3, 1, 1)
plt.plot(t_num, x_t(t_num, A, T, d1, d2))
plt.title(f'Señal x(t)  (A={A}, T={T}, d1={d1}, d2={d2})')
plt.xlabel('Tiempo [s]')
plt.ylabel('x(t)')
plt.grid(True)
plt.xlim(-T/2, T/2)

# 2️⃣ Magnitud del espectro
plt.subplot(3, 1, 2)
plt.stem(n_values, X_exact_magn, linefmt='b-', markerfmt='bo', basefmt='r-')
plt.stem(n_values, np.abs(X_numerical), linefmt='r--', markerfmt='rx', basefmt='r-')
plt.title('Magnitud del Espectro |X[n]| (Azul = Exacto, Rojo = Numérico)')
plt.xlabel('Índice de Frecuencia n')
plt.ylabel('|X[n]|')
plt.grid(True)

# 3️⃣ Fase del espectro
plt.subplot(3, 1, 3)
plt.stem(n_values, X_exact_phase, linefmt='b-', markerfmt='bo', basefmt='r-')
plt.title('Fase del Espectro φₙ')
plt.xlabel('Índice de Frecuencia n')
plt.ylabel('Fase [rad]')
plt.yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
           ['$-π$', '$-π/2$', '0', '$π/2$', '$π$'])
plt.ylim(-1.1*np.pi, 1.1*np.pi)
plt.grid(True)

plt.tight_layout()
plt.show()

1. **Señal x(t):** se observan las rampas que definen un periodo de la señal.  
2. **Magnitud del espectro:** muestra cómo la energía se concentra en los primeros armónicos.  
3. **Fase del espectro:** indica la simetría y los desfases relativos entre armónicos.

El método analítico y el numérico deben coincidir casi exactamente, validando el cálculo teórico.
